In [4]:
import pandas as pd
import numpy as np 
import requests
from ALAN.data.data_engineering import get_location, distance_calculation

In [5]:
#to get details from a certain stop: 
url = 'https://v5.bvg.transport.rest/locations?'
params = {'poi' : 'false', 
          'adresses' : 'false',
          'query' : 'mehringdamm'
         }

In [45]:
data = requests.get(url,params)

In [46]:
data.json()

[{'type': 'stop',
  'id': '900000017101',
  'name': 'U Mehringdamm',
  'location': {'type': 'location',
   'id': '900017101',
   'latitude': 52.49357,
   'longitude': 13.388138},
  'products': {'suburban': False,
   'subway': True,
   'tram': False,
   'bus': True,
   'ferry': False,
   'express': False,
   'regional': False},
  'stationDHID': 'de:11000:900017101'},
 {'type': 'stop',
  'id': '900000017100',
  'name': 'Obentrautstr./U Mehringdamm',
  'location': {'type': 'location',
   'id': '900017100',
   'latitude': 52.49535,
   'longitude': 13.389612},
  'products': {'suburban': False,
   'subway': False,
   'tram': False,
   'bus': True,
   'ferry': False,
   'express': False,
   'regional': False},
  'stationDHID': 'de:11000:900017100'},
 {'type': 'location',
  'id': None,
  'latitude': 52.490334,
  'longitude': 13.386421,
  'address': '10961 Berlin-Kreuzberg, Mehringdamm'},
 {'type': 'location',
  'id': None,
  'latitude': 52.486073,
  'longitude': 13.386017,
  'address': '10965 

In [82]:
location = get_location('Obentrautstraße 71')

In [83]:
latitude = round(location[0],3)
longitude = round(location[1],3)

In [130]:
url_nearby = 'https://v5.bvg.transport.rest/stops/nearby?'
params_ = {'latitude' : latitude,
          'longitude' : longitude
         }

In [131]:
data_1 = requests.get(url_nearby,params_)

In [133]:
query = data_1.json()

In [263]:
query

[{'type': 'stop',
  'id': '900000017104',
  'name': 'U Möckernbrücke',
  'location': {'type': 'location',
   'id': '900017104',
   'latitude': 52.498945,
   'longitude': 13.383257},
  'products': {'suburban': False,
   'subway': True,
   'tram': False,
   'bus': True,
   'ferry': False,
   'express': False,
   'regional': False},
  'stationDHID': 'de:11000:900017104',
  'distance': 264},
 {'type': 'stop',
  'id': '900000017105',
  'name': 'Hornstr',
  'location': {'type': 'location',
   'id': '900017105',
   'latitude': 52.493633,
   'longitude': 13.383823},
  'products': {'suburban': False,
   'subway': False,
   'tram': False,
   'bus': True,
   'ferry': False,
   'express': False,
   'regional': False},
  'stationDHID': 'de:11000:900017105',
  'distance': 420},
 {'type': 'stop',
  'id': '900000017103',
  'name': 'U Gleisdreieck',
  'location': {'type': 'location',
   'id': '900017103',
   'latitude': 52.499583,
   'longitude': 13.374295},
  'products': {'suburban': False,
   'subway

In [264]:
df_name = []
df_lat = []
df_lon = []
df_dist = []
types = []

In [265]:
keys = list(query[i]['products'].keys())

In [266]:
for i in range(len(query)): 
    df_name.append(query[i]['name'])
    df_lat.append(query[i]['location']['latitude'])
    df_lon.append(query[i]['location']['longitude'])
    df_dist.append(query[i]['distance'])


In [267]:
for i in range(len(query)): 
    for x in range(len(keys)): 
        if query[i]['products'][keys[x]] == True: 
            print(query[i]['name'])
            types.append(keys[x])
            print(types)

U Möckernbrücke
['subway']
U Möckernbrücke
['subway', 'bus']
Hornstr
['subway', 'bus', 'bus']
U Gleisdreieck
['subway', 'bus', 'bus', 'subway']
Mehringbrücke
['subway', 'bus', 'bus', 'subway', 'bus']
Willy-Brandt-Haus
['subway', 'bus', 'bus', 'subway', 'bus', 'bus']
Obentrautstr./U Mehringdamm
['subway', 'bus', 'bus', 'subway', 'bus', 'bus', 'bus']
U Mehringdamm
['subway', 'bus', 'bus', 'subway', 'bus', 'bus', 'bus', 'subway']
U Mehringdamm
['subway', 'bus', 'bus', 'subway', 'bus', 'bus', 'bus', 'subway', 'bus']
U Hallesches Tor
['subway', 'bus', 'bus', 'subway', 'bus', 'bus', 'bus', 'subway', 'bus', 'subway']
U Hallesches Tor
['subway', 'bus', 'bus', 'subway', 'bus', 'bus', 'bus', 'subway', 'bus', 'subway', 'bus']


In [273]:
df = pd.DataFrame()

In [274]:
df['name'] = df_name
df['lat'] = df_lat
df['lon'] = df_lon
df['dist'] = df_dist

In [275]:
df['walking time'] = df['dist'].apply(lambda x: (x*60)/7000)

In [276]:
df

,name,lat,lon,dist,walking time
0,U Möckernbrücke,52.498945,13.383257,264,2.262857
1,Hornstr,52.493633,13.383823,420,3.600000
2,U Gleisdreieck,52.499583,13.374295,537,4.602857
3,Mehringbrücke,52.497974,13.389262,569,4.877143
4,Willy-Brandt-Haus,52.500411,13.387437,577,4.945714
5,Obentrautstr./U Mehringdamm,52.495350,13.389612,611,5.237143
6,U Mehringdamm,52.493570,13.388138,616,5.280000
7,U Hallesches Tor,52.497777,13.391761,733,6.282857


In [312]:
latitude = 52.52446
longitude = 13.40812
base = 'https://v5.bvg.transport.rest/stops/reachable-from?'
params_1 = {'latitude' : latitude,
            'longitude' : longitude,
            'address' : '10178+Berlin-Mitte,+Münzstr.+12',
            'maxDuration' : 5
           }

In [289]:
query_1 = requests.get(base,params_1).json()

In [322]:
a = [[1,2,4],[5,6,7], [8,9,10]]


In [326]:
for i in range(len(a)): 
    for x in range(0,3,1): 
        print(a[i][x])

1
2
4
5
6
7
8
9
10


In [335]:
query_1[0]['duration']

3

In [340]:
query_1[0]['stations'][0]

{'type': 'stop',
 'id': '900000100030',
 'name': 'Memhardstr.',
 'location': {'type': 'location',
  'id': '900100030',
  'latitude': 52.524034,
  'longitude': 13.41098},
 'products': {'suburban': True,
  'subway': True,
  'tram': True,
  'bus': True,
  'ferry': False,
  'express': False,
  'regional': True},
 'stationDHID': 'de:11000:900100030'}

In [298]:
duration = []
stations = []
for i in range(len(query_1)):
    stations.append(query_1[i]['stations']['name'])
    duration.append(query_1[i]['duration'])

TypeError: list indices must be integers or slices, not str

In [295]:
duration

[3, 3, 3]